## Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install Requirements

In [1]:
!pip install --upgrade deepface
!pip install --upgrade tensorflow

## Import Library

In [2]:
import os
import pickle
import numpy as np
from tqdm import tqdm
from deepface import DeepFace
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [3]:
# Path dataset dan hasil ekstraksi
path_val = "PROJEK_STKI/DatasetFinal/val"
path_output = "PROJEK_STKI/DatasetFinal/model"

# Pastikan folder output ada
os.makedirs(path_output, exist_ok=True)

In [5]:
# Inisialisasi model
model_name = "VGG-Face"

# Inisialisasi model terlebih dahulu
DeepFace.build_model(model_name)

# Fungsi untuk ekstraksi fitur menggunakan DeepFace
def ekstraksi_fitur_val(folder_path):
    fitur = []
    label = []

    for kategori in ["Drowsy", "Non Drowsy"]:
        kategori_path = os.path.join(folder_path, kategori)
        for subfolder in os.listdir(kategori_path):
            subfolder_path = os.path.join(kategori_path, subfolder)
            for file in tqdm(os.listdir(subfolder_path), desc=f"Ekstraksi {kategori}/{subfolder}"):
                if file.endswith(".png"):
                    file_path = os.path.join(subfolder_path, file)
                    try:
                        # Ekstraksi fitur
                        embedding = DeepFace.represent(img_path=file_path, model_name=model_name, enforce_detection=False)
                        fitur.append(embedding[0]["embedding"])
                        label.append(kategori)
                        print(f"{file} pada folder {subfolder} dilakukan ekstraksi fitur dengan menggunakan {model_name}.")
                    except Exception as e:
                        print(f"Gagal memproses {file}: {e}")
    return np.array(fitur), np.array(label)

# Ekstraksi fitur dari folder val
print("Memulai ekstraksi fitur dari folder val...")
fitur_val, label_val = ekstraksi_fitur_val(path_val)

# Simpan hasil ekstraksi ke file pickle
output_file_val = os.path.join(path_output, "fitur_val_vggface.pkl")
with open(output_file_val, "wb") as f:
    pickle.dump({"features": fitur_val, "labels": label_val}, f)
print(f"Hasil ekstraksi disimpan di {output_file_val}")

Memulai ekstraksi fitur dari folder val...


Ekstraksi Drowsy/X:   8%|█████▎                                                         | 1/12 [00:06<01:14,  6.74s/it]

X0001.png pada folder X dilakukan ekstraksi fitur dengan menggunakan VGG-Face.


Ekstraksi Drowsy/X:  17%|██████████▌                                                    | 2/12 [00:08<00:35,  3.60s/it]

X0005.png pada folder X dilakukan ekstraksi fitur dengan menggunakan VGG-Face.


Ekstraksi Drowsy/X:  25%|███████████████▊                                               | 3/12 [00:09<00:23,  2.66s/it]

X0008.png pada folder X dilakukan ekstraksi fitur dengan menggunakan VGG-Face.


Ekstraksi Drowsy/X:  25%|███████████████▊                                               | 3/12 [00:10<00:31,  3.50s/it]


KeyboardInterrupt: 

In [ ]:
# Visualisasi menggunakan t-SNE
print("Memulai visualisasi t-SNE...")
tsne = TSNE(n_components=2, random_state=42)
fitur_2d = tsne.fit_transform(fitur_val)

# Plot hasil t-SNE
plt.figure(figsize=(10, 8))
for kategori in ["Drowsy", "Non Drowsy"]:
    idx = np.array(label_val) == kategori
    plt.scatter(fitur_2d[idx, 0], fitur_2d[idx, 1], label=kategori, alpha=0.7)

plt.title("Visualisasi t-SNE Hasil Ekstraksi Fitur (Val)")
plt.xlabel("Dimensi 1")
plt.ylabel("Dimensi 2")
plt.legend(loc="best")
plt.grid(True)
plt.show()